## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

## Finding Chessboard Corners to Calibrate Camera

In [2]:
# Prepare object points, ex: (0,0,0), (1,0,0), (2,0,0), ..., (8,5,0)
objp = np.zeros((9*6,3),np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store the object points and image points from all the calibration images.
objpts = [] # 3d points in real world space
imgpts = [] # 2d points in image plane

# Make a list of the calibration images
cal_imgs = glob.glob('camera_cal/calibration*.jpg')

# Cycle through the list and perform chessboard corner search
for fname in cal_imgs:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_size = gray.shape[::-1]
    
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
    
    # If found, add object points and image points to arrays
    if ret:
        objpts.append(objp)
        imgpts.append(corners)
        
        # Draw and display the corners on calibration images
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# Calibrate camera and undistort images

In [6]:
# Calibrate camera using the image points generated from previous step
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpts, imgpts, img_size, None, None)

# Read in the test images.
straightline_imgs = glob.glob('test_images/straight_lines*.jpg')
undist_imgs = []

# Cycle the imgs to output undistorted ones.
for fname in straightline_imgs:
    img = cv2.imread(fname)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    undist_imgs.append(undist)

cv2.imshow('test', undist_imgs[0])
cv2.waitKey()
cv2.destroyAllWindows()

## Create Threshold Binary Images using undistorted images

In [ ]:
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0,255)):
    xorder = 1
    yorder = 0
    if orient=='y':
        xorder = 0
        yorder = 1
    sobel = cv2.Sobel(img, cv2.CV_64F, xorder, yorder, ksize=sobel_kernel)
    abs_sobel = np.absolute(sobel)
    grad_binary = np.zeros_like(abs_sobel)
    grad_binary[(abs_sobel>thresh[0]) & (abs_sobel<thresh[1])] = 1
    return grad_binary

def mag_thresh(img, sobel_kernel=3, thresh=(0,255)):
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    mag_sobel = np.sqrt(np.square(sobelx) + np.square(sobely))
    mag_binary = np.zeros_like(mag_sobel)
    mag_binary[(mag_sobel>thresh[0]) & (mag_sobel<thresh[1])] = 1
    return mag_binary

def dir_thresh(img, sobel_kernel=3, thresh=(0,255)):
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    dir_sobel = np.arctan2(abs_sobely, abs_sobelx)
    dir_binary = np.zeros_like(dir_sobel)
    dir_binary[(dir_sobel>thresh[0]) & (dir_sobel<thresh[1])] = 1
    return dir_binary

#image = cv2.imread('test_images/signs_vehicles_xygrad.png')
#gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

gray = cv2.cvtColor(undist_imgs[0], cv2.COLOR_BGR2GRAY)

ksize = 5
thresh = (90,180)

gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=ksize, thresh=thresh)
grady = abs_sobel_thresh(gray, orient='y', sobel_kernel=ksize, thresh=thresh)
mag_binary = mag_thresh(gray, sobel_kernel=ksize, thresh=thresh)
dir_binary = dir_thresh(gray, sobel_kernel=ksize, thresh=(0, np.pi/2))

combined = np.zeros_like(dir_binary)
combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1

cv2.imshow('output', combined)
cv2.waitKey()
cv2.destroyAllWindows()
